In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import data_processing.load_sqlite as load_sql

## Predicting Success with Frequency Encoding

In [2]:
df = load_sql.fetch_movie_rating_features()
df.head()

,movie_id,vote_average,title,release_date,runtime,top_5_cast_ids,num_cast_members,director_ids,writer_ids,company_ids,genre_ids
0,2,7.113,Ariel,1988-10-21,73,None,0,None,None,2303,"18,35,80,10749"
1,3,7.300,Shadows in Paradise,1986-10-17,74,None,0,None,None,2303,"18,35,10749"
2,5,5.858,Four Rooms,1995-12-09,98,"3129,3130,3131,3124,2555",28,"138,2294,3110,3111","138,2294,3110,3111","14,59","35,80"
3,6,6.500,Judgment Night,1993-10-15,109,"2880,9777,5724,10822,12799",31,2042,"52035,52035,1088307","33,182,1644","28,53,80"
4,11,8.200,Star Wars,1977-05-25,121,"2,3,4,5,12248",104,1,1,"1,25","12,28,878"


In [3]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26204 entries, 2 to 44520
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   movie_id          26204 non-null  int64  
 1   vote_average      26204 non-null  float64
 2   title             26204 non-null  object 
 3   release_date      26204 non-null  object 
 4   runtime           26204 non-null  int64  
 5   top_5_cast_ids    26204 non-null  object 
 6   num_cast_members  26204 non-null  int64  
 7   director_ids      26204 non-null  object 
 8   writer_ids        26204 non-null  object 
 9   company_ids       26204 non-null  object 
 10  genre_ids         26204 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 2.4+ MB


In [4]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'] = df['release_date'].astype(int) / 10**9  # Unix timestamp in seconds

In [6]:
df["successful"] = (df["vote_average"] > 7.0).astype(int) 
df.columns

Index(['movie_id', 'vote_average', 'title', 'release_date', 'runtime',
       'top_5_cast_ids', 'num_cast_members', 'director_ids', 'writer_ids',
       'company_ids', 'genre_ids', 'successful'],
      dtype='object')

In [8]:
df.corr(numeric_only=True)["successful"].sort_values(ascending=False)

successful          1.000000
vote_average        0.643150
runtime             0.157104
num_cast_members    0.100434
movie_id            0.037698
release_date       -0.000986
Name: successful, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split

# Split the dataset first
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.columns

Index(['movie_id', 'vote_average', 'title', 'release_date', 'runtime',
       'top_5_cast_ids', 'num_cast_members', 'director_ids', 'writer_ids',
       'company_ids', 'genre_ids', 'successful'],
      dtype='object')

In [10]:
categorical_cols = ['company_ids', 'top_5_cast_ids', 'director_ids', 'writer_ids']

for col in categorical_cols:
    # Compute frequencies in training data
    freq_map = train_df[col].value_counts().to_dict()
    
    # Map frequencies in both training and test sets
    train_df[col + '_freq'] = train_df[col].map(freq_map)
    test_df[col + '_freq'] = test_df[col].map(freq_map)

    # Fill missing values with a default frequency (e.g., 1)
    train_df[col + '_freq'] = train_df[col + '_freq'].fillna(1)
    test_df[col + '_freq'] = test_df[col + '_freq'].fillna(1)

train_df.corr(numeric_only=True)["successful"].sort_values(ascending=False)

successful             1.000000
vote_average           0.643328
runtime                0.163982
num_cast_members       0.095581
top_5_cast_ids_freq    0.050896
director_ids_freq      0.048338
movie_id               0.032802
writer_ids_freq        0.032210
company_ids_freq       0.006549
release_date           0.000718
Name: successful, dtype: float64

In [11]:
def one_hot_encode_genres(df):
    """One-hot encodes the genre_ids column."""
    # Convert genre_ids to lists
    df["genre_ids"] = df["genre_ids"].apply(lambda x: x.split(','))
    
    # Extract unique genre IDs
    all_genres = sorted(set(genre for genres in df["genre_ids"] for genre in genres))

    # Create one-hot encoded columns
    for genre in all_genres:
        df[f"genre_{genre}"] = df["genre_ids"].apply(lambda x: int(genre in x))

    # Drop original genre_ids column
    df.drop(columns=["genre_ids"], inplace=True)
    return df

# Apply to both train and test sets
train_df = one_hot_encode_genres(train_df)
test_df = one_hot_encode_genres(test_df)

print(train_df.columns)
print(test_df.columns)

Index(['movie_id', 'vote_average', 'title', 'release_date', 'runtime',
       'top_5_cast_ids', 'num_cast_members', 'director_ids', 'writer_ids',
       'company_ids', 'successful', 'company_ids_freq', 'top_5_cast_ids_freq',
       'director_ids_freq', 'writer_ids_freq', 'genre_10402', 'genre_10749',
       'genre_10751', 'genre_10752', 'genre_10770', 'genre_12', 'genre_14',
       'genre_16', 'genre_18', 'genre_27', 'genre_28', 'genre_35', 'genre_36',
       'genre_37', 'genre_53', 'genre_80', 'genre_878', 'genre_9648',
       'genre_99'],
      dtype='object')
Index(['movie_id', 'vote_average', 'title', 'release_date', 'runtime',
       'top_5_cast_ids', 'num_cast_members', 'director_ids', 'writer_ids',
       'company_ids', 'successful', 'company_ids_freq', 'top_5_cast_ids_freq',
       'director_ids_freq', 'writer_ids_freq', 'genre_10402', 'genre_10749',
       'genre_10751', 'genre_10752', 'genre_10770', 'genre_12', 'genre_14',
       'genre_16', 'genre_18', 'genre_27', 'genre_28'

In [12]:
# Select features to keep (numeric columns)
features_to_keep = train_df.select_dtypes(include=["float64", "int64"]).columns

# Prepare the training and test data
X_train = train_df[features_to_keep].drop(columns=["vote_average", "successful"])
y_train = train_df["successful"]
X_test = test_df[features_to_keep].drop(columns=["vote_average", "successful"])
y_test = test_df["successful"]

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Define and train the classifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
print("Training score:", rf.score(X_train, y_train))
print("Testing score:", rf.score(X_test, y_test))

Training score: 0.9997137814244145
Testing score: 0.8227437511925205


## Predict Success using Categorical Scores

In [2]:
lambda_director   = 0.000408
lambda_writers    = 0.000377
lambda_cast_time  = 0.000592
lambda_cast_order = 0.122

df = load_sql.fetch_predict_success_data(lambda_director, lambda_writers, lambda_cast_time, lambda_cast_order)
df.head()

,movie_id,vote_average,runtime,num_cast_members,release_date,Adventure,Fantasy,Animation,Drama,Horror,...,Music,Romance,Family,War,TV Movie,director_score,writer_score,cast_score,production_company_score,successful
0,2,7.113,73,0,593395200.0,0,0,0,1,0,...,0,1,0,0,0,6.292833,6.292833,6.292833,6.687600,1
1,3,7.300,74,0,529891200.0,0,0,0,1,0,...,0,1,0,0,0,6.292833,6.292833,6.292833,6.512667,1
2,5,5.858,98,28,818467200.0,0,0,0,0,0,...,0,0,0,0,0,7.333995,7.355491,6.648418,6.302417,0
3,6,6.500,109,31,750643200.0,0,0,0,0,0,...,0,0,0,0,0,5.882887,6.119896,6.370170,6.376285,0
4,11,8.200,121,104,233366400.0,1,0,0,0,0,...,0,0,0,0,0,6.782933,6.676182,6.170755,6.623217,1


In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
# Define features
X_train = train_df.drop(columns=['vote_average', 'movie_id', 'successful'])
X_test = test_df.drop(columns=['vote_average', 'movie_id', 'successful'])

# Define target
y_train = train_df['successful']
y_test = test_df['successful']

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Define and train the classifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
print("Training score:", rf.score(X_train, y_train))
print("Testing score:", rf.score(X_test, y_test))

Training score: 0.9996630822360109
Testing score: 0.815272318921954


In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=50, random_state=42)
gb.fit(X_train, y_train)

print(gb.score(X_train, y_train))
print(gb.score(X_test, y_test))

0.8139090883566836
0.8096574957888827
